In [15]:
%load_ext autoreload
%autoreload 2
%pylab inline

from collections import defaultdict, OrderedDict, Counter, namedtuple
import copy
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from datetime import date, datetime
import os
from tputils import *
import json
import requests

from onmt.utils.postprocess import postprocess_texts

pd.set_option('max_colwidth',130)
matplotlib.rcParams['figure.figsize'] = (14, 8)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/home/tomas/.pyenv/versions/3.7.0/envs/nmt/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text', 'datetime', 'random', 'copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
test_files = {
    "src": "OpenSubtitles.cs-en.cs.2.tok.test",
    "tgt": "OpenSubtitles.cs-en.en.2.tok.test",
    "pred": "os2.test.pred.txt",
}

base_path = ".."

texts_dict = {}
for key, filename in test_files.items():
    texts = txt_load(os.path.join(base_path, filename))
    texts = postprocess_texts(texts)
    texts_dict[key] = texts
    
    
pd.DataFrame(texts_dict)

,src,tgt,pred
0,"nalezení života, dokonce i primitivního mikrobiálního života, by změnilo naše místo ve vesmíru tím, že nám řekne, že nejsme sami.","finding life, even primitive microbial life, would change our place in the universe by telling us that we're not alone.","finding life, even primitivního mikrobiálního life would change our space in the universe that we're not alone."
1,"a to bude s největší pravděpodobností znamení, že jsme stále blíže k nalezení inteligentní civilizace.",and it would most likely be an indication that we're getting closer to finding an intelligent civilisation.,and that will be the most likely sign that we're still closer to finding intelligent civilization.
2,"bylo by opravdu zábavné, kdybychom mohli najít cizince kteří by nám mohli poslat e - mail nebo být na twitteru, nebo cokoli ji...","it would be really fun if we could find an alien that could, you know, send me an email or twitter or whatever, all day long, ...","it would really be fun if we could find a stranger who could send us the e - mail or to be on the twitteru or anything else, b..."
3,"jakmile najdeme mikrobiální, je šance na nalezení inteligence.","once we have the microbial, we have the potential for intelligence.","once we find mikrobiální, there's a chance to find an intelligence."
4,"ale pokud nenajdeme vůbec žádnou formu života, nemáme potenciál pro nic.","but if we don't have a life form at all, we don't have the potential for anything.","but if we don't find any form of life, we don't have potential for nothing."
5,co kdybychom našli složitý život tam venku?,what if we do find complex life out there?,what if we find a complicated life out there?
6,jak by vypadal?,what would it look like?,how would he look?
7,"bylo by to něco, jako to, co vidíme v hollywoodských filmech?",would it be anything like what we see in hollywood movies?,would it be like what we see in hollywoodských movies?
8,"je legrační, že hollywood zobrazuje mimozemšťany více či méně stejným způsobem.",it is humorous that hollywood depicts aliens more or less the same way.,it's funny that hollywood is showing aliens more or less in the same way.
9,"je tu klingon a tam romulan a tam jsou zešikmené oči a víte, velké staré mozky a tak dále.","there's a klingon here and a romulan there and there's slanty eyes and, you know, big old brains and so on.","there's a klingon, and there's a romulan and there's zešikmené eyes and you know, big old brains and so come on."


In [23]:
import re

texts = [
    "ad . dfa", 
    " dfji ? dad", 
    "ad fas aaa"
]

for text in texts:
    print(re.sub(" ([.,?!;])", r"\1", text))

ad. dfa
 dfji? dad
ad fas aaa


In [19]:
import pyonmttok
tokenizer = pyonmttok.tokenizer.Tokenizer("conservative", case_feature=True)

texts = txt_load("../data/test_cs.txt")

for text in texts:
    tokens, features = tokenizer.tokenize(text)
    print(tokens)
    print(features)
    print()

['bílá', 'barva', 'je', 'má', 'oblíbená']
[['L', 'L', 'L', 'L', 'L']]

['bílá', 'barva', 'v', 'plechovce']
[['L', 'L', 'L', 'L']]

['kráva', 'má', 'ostrý', 'roh']
[['L', 'L', 'L', 'L']]

['roh', 'místnosti', 'je', 'tmavý']
[['L', 'L', 'L', 'L']]

['přijdu', 'brzy']
[['L', 'L']]

['vstávám', 'brzy', 'ráno']
[['L', 'L', 'L']]

['má', 'velké', 'kozy']
[['L', 'L', 'L']]

['viděl', 'jsem', 'ovce', ',', 'kozy', 'a', 'jiný', 'dobytek']
[['L', 'L', 'L', 'N', 'L', 'L', 'L', 'L']]

['vůbec', 'nevíme', ',', 'jak', 'chce', 'kremlík', 'urychlit', 'výstavbu', 'dálnic', ',', 'nebo', 'jestli', 'chce', 'zdražit', 'mýtné', '.']
[['C', 'L', 'N', 'L', 'L', 'C', 'L', 'L', 'L', 'N', 'L', 'L', 'L', 'L', 'L', 'N']]

['estonsko', 'a', 'polsko', 'zakázaly', 'vplout', 'do', 'svých', 'vod', 'ruské', 'školní', 'plachetnici']
[['C', 'L', 'C', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L']]

['další', 'dvě', 'italská', 'městečka', 'nabízejí', 'domy', 'za', 'euro']
[['C', 'L', 'L', 'L', 'L', 'L', 'L', 'L']]

['vozíčkář', 's